# Auditoría de Cotizaciones - Radii
## Reto Técnico - Imanol Muñiz

Este notebook contiene la auditoría completa del batch de 10 cotizaciones generadas por IA.

In [8]:
import numpy as np
import pandas as pd
import math

# Densidades de materiales (g/cm³)
DENSIDADES = {
    'Aluminum 6061': 2.70,
    'Aluminum 7075': 2.81,
    'Steel 1018': 7.85,
    'Steel 4140': 7.85,
    'Titanium': 4.43,
    'Stainless 304': 8.00,
    'Stainless 316': 8.00
}

In [9]:
def redondear_stock(dimensiones):
    return [math.ceil(dim / 10) * 10 for dim in dimensiones]

def calcular_peso_caja(largo, ancho, alto, densidad):
    # Redondear stock
    l_stock, a_stock, h_stock = redondear_stock([largo, ancho, alto])
    # Volumen en mm³ -> cm³
    volumen_cm3 = (l_stock * a_stock * h_stock) / 1000    
    # Peso en g -> kg
    peso_kg = (volumen_cm3 * densidad) / 1000    
    return peso_kg, l_stock, a_stock, h_stock

def calcular_peso_cilindro(diametro, longitud, densidad):
    # Redondear stock
    d_stock, l_stock = redondear_stock([diametro, longitud])
    # Radio en cm
    radio_cm = d_stock / 20  # Convertir de mm a cm y dividir por 2
    longitud_cm = l_stock / 10
    # Volumen en cm³
    volumen_cm3 = math.pi * (radio_cm ** 2) * longitud_cm
    # Peso en g -> kg
    peso_kg = (volumen_cm3 * densidad) / 1000
    return peso_kg, d_stock, l_stock

def calcular_peso_cilindro_hueco(diametro_ext, espesor_pared, longitud, densidad):
    # Redondear stock (solo el diámetro externo y longitud)
    d_ext_stock, l_stock = redondear_stock([diametro_ext, longitud])
    # Diámetro interno
    d_int = d_ext_stock - (2 * espesor_pared)
    # Radios en cm
    r_ext_cm = d_ext_stock / 20
    r_int_cm = d_int / 20
    longitud_cm = l_stock / 10
    # Volumen en cm³
    volumen_cm3 = math.pi * ((r_ext_cm ** 2) - (r_int_cm ** 2)) * longitud_cm
    # Peso en g -> kg
    peso_kg = (volumen_cm3 * densidad) / 1000
    return peso_kg, d_ext_stock, d_int, l_stock

# Funciones auxiliares para el dataframe
def procesar_caja_df(dims_str, densidad, precio_kg):
    """Procesa una caja desde string de dimensiones del dataframe"""
    dims = dims_str.replace(' ', '').split('×')
    largo, ancho, alto = [float(d) for d in dims]
    peso_kg, l_stock, a_stock, h_stock = calcular_peso_caja(largo, ancho, alto, densidad)
    dims_redondeadas = f"{int(l_stock)} × {int(a_stock)} × {int(h_stock)}"
    precio_material = peso_kg * precio_kg
    return dims_redondeadas, peso_kg, precio_material

def procesar_cilindro_df(dims_str, densidad, precio_kg):
    """Procesa un cilindro desde string de dimensiones del dataframe"""
    dims = dims_str.replace('Ø', '').replace(' ', '').split('×')
    diametro, longitud = [float(d) for d in dims]
    peso_kg, d_stock, l_stock = calcular_peso_cilindro(diametro, longitud, densidad)
    dims_redondeadas = f"Ø{int(d_stock)} × {int(l_stock)}"
    precio_material = peso_kg * precio_kg
    return dims_redondeadas, peso_kg, precio_material

def procesar_cilindro_hueco_df(dims_str, densidad, precio_kg):
    """Procesa un cilindro hueco desde string de dimensiones del dataframe"""
    # Formato: "Ø60 (p5) × 150"
    # Extraer diámetro externo
    d_ext = float(dims_str.split('(')[0].replace('Ø', '').replace(' ', ''))
    # Extraer espesor de pared
    espesor = float(dims_str.split('(p')[1].split(')')[0])
    # Extraer longitud
    longitud = float(dims_str.split('×')[1].replace(' ', ''))
    
    peso_kg, d_ext_stock, d_int_calc, l_stock = calcular_peso_cilindro_hueco(d_ext, espesor, longitud, densidad)
    dims_redondeadas = f"Ø{int(d_ext_stock)} (p{(int(d_ext_stock) - int(d_int_calc)) / 2}) × {int(l_stock)}"
    precio_material = peso_kg * precio_kg
    return dims_redondeadas, peso_kg, precio_material

In [10]:
import pandas as pd

# Crear el dataframe con los datos de la tabla
data = {
    'ID': ['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8', 'Q9', 'Q10'],
    'Cliente': ['Acme Corp', 'Acme Corp', 'BetaTech', 'BetaTech', 'CanDo Mfg', 
                'CanDo Mfg', 'DeltaAero', 'EastMech', 'EastMech', 'Acme Corp'],
    'Material': ['Aluminum 6061', 'Aluminum 6061', 'Steel 4140', 'Steel 1018', 
                 'Titanium', 'Titanium', 'Aluminum 7075', 'Stainless 316', 
                 'Stainless 304', 'Aluminum 6061'],
    'Forma': ['Caja', 'Caja', 'Cilindro', 'Cilindro', 'Caja', 'Caja', 'Caja', 
              'Cilindro hueco', 'Cilindro hueco', 'Caja'],
    'Dimensiones (mm)': ['102 × 98 × 47', '102 × 98 × 47', 'Ø80 × 200', 'Ø80 × 200',
                          '2 × 2 × 4', '50 × 50 × 100', '203 × 152 × 22',
                          'Ø60 (p5) × 150', 'Ø60 (p5) × 150', '102 × 98 × 47'],
    'Qty': [1, 25, 5, 5, 1, 20, 50, 10, 10, 1],
    'Peso IA (kg)': [1.26, 1.26, 2.70, 7.90, 1.10, 1.10, 2.10, 2.00, 2.00, 1.26],
    'Precio/kg (USD)': [7.50, 7.50, 4.75, 2.25, 62.00, 62.00, 15.00, 9.50, 6.50, 7.50],
    'Precio Total IA': [185, 185, 290, 285, 320, 145, 95, 175, 210, 140],
    'Lead Time': ['10 días', '10 días', '7 días', '7 días', '14 días', '14 días', 
                  '3 días', '8 días', '8 días', '3 días']
}

df_cotizaciones = pd.DataFrame(data)

In [11]:
# Agregar columnas al dataframe
dimensiones_redondeadas = []
pesos_calculados = []
costos_material = []
costos_negocio = []
costo_negocio_IA = []

for idx, row in df_cotizaciones.iterrows():
    forma = row['Forma']
    dims = row['Dimensiones (mm)']
    material = row['Material']
    precio_kg = row['Precio/kg (USD)']
    qty = row['Qty']
    lead_time = row['Lead Time']
    precio_IA = row['Precio Total IA']
    
    # Obtener densidad del material
    densidad = DENSIDADES.get(material, 2.7)  # Default a aluminum si no se encuentra
    
    # Procesar según la forma usando las funciones de la celda 3
    if forma == 'Caja':
        dims_redondeadas, peso_kg, costo_mat = procesar_caja_df(dims, densidad, precio_kg)
    elif forma == 'Cilindro':
        dims_redondeadas, peso_kg, costo_mat = procesar_cilindro_df(dims, densidad, precio_kg)
    elif forma == 'Cilindro hueco':
        dims_redondeadas, peso_kg, costo_mat = procesar_cilindro_hueco_df(dims, densidad, precio_kg)
    else:
        dims_redondeadas, peso_kg, costo_mat = dims, 0, 0
    
    costos_material.append(round(costo_mat, 2))
    costo_neg = costo_mat * 1.15
    # 1. Descuento del 17% si Qty > 10
    if qty > 10:
        costo_neg = costo_neg * 0.83  # Aplicar descuento del 17%
    
    # 2. Aumento del 25% si lead time < 5 días
    lead_time_dias = int(lead_time.split()[0])  # Extraer número de días
    if lead_time_dias < 5:
        costo_neg = costo_neg * 1.25  # Aumentar 25%
    
    # 3. Si es Titanium, agregar 150 USD / Qty
    if 'Titanium' in material:
        costo_neg = costo_neg + (150 / qty)
    
    costo_neg_IA = precio_IA * 0.77

    dimensiones_redondeadas.append(dims_redondeadas)
    pesos_calculados.append(round(peso_kg, 2))
    costos_negocio.append(round(costo_neg, 2))
    costo_negocio_IA.append(round(costo_neg_IA, 2))

# Agregar las columnas al dataframe
df_cotizaciones['Dimensiones redondeadas'] = dimensiones_redondeadas
df_cotizaciones['Peso calculado (kg)'] = pesos_calculados
df_cotizaciones['Costo material (USD)'] = costos_material
df_cotizaciones['Costo negocio sin mano de obra (USD)'] = costos_negocio
# df_cotizaciones['Costo negocio IA (USD)'] = costo_negocio_IA

df_cotizaciones

,ID,Cliente,Material,Forma,Dimensiones (mm),Qty,Peso IA (kg),Precio/kg (USD),Precio Total IA,Lead Time,Dimensiones redondeadas,Peso calculado (kg),Costo material (USD),Costo negocio sin mano de obra (USD)
0,Q1,Acme Corp,Aluminum 6061,Caja,102 × 98 × 47,1,1.26,7.50,185,10 días,110 × 100 × 50,1.49,11.14,12.81
1,Q2,Acme Corp,Aluminum 6061,Caja,102 × 98 × 47,25,1.26,7.50,185,10 días,110 × 100 × 50,1.49,11.14,10.63
2,Q3,BetaTech,Steel 4140,Cilindro,Ø80 × 200,5,2.70,4.75,290,7 días,Ø80 × 200,7.89,37.49,43.11
3,Q4,BetaTech,Steel 1018,Cilindro,Ø80 × 200,5,7.90,2.25,285,7 días,Ø80 × 200,7.89,17.76,20.42
4,Q5,CanDo Mfg,Titanium,Caja,2 × 2 × 4,1,1.10,62.00,320,14 días,10 × 10 × 10,0.00,0.27,150.32
5,Q6,CanDo Mfg,Titanium,Caja,50 × 50 × 100,20,1.10,62.00,145,14 días,50 × 50 × 100,1.11,68.66,73.04
6,Q7,DeltaAero,Aluminum 7075,Caja,203 × 152 × 22,50,2.10,15.00,95,3 días,210 × 160 × 30,2.83,42.49,50.69
7,Q8,EastMech,Stainless 316,Cilindro hueco,Ø60 (p5) × 150,10,2.00,9.50,175,8 días,Ø60 (p5.0) × 150,1.04,9.85,11.33
8,Q9,EastMech,Stainless 304,Cilindro hueco,Ø60 (p5) × 150,10,2.00,6.50,210,8 días,Ø60 (p5.0) × 150,1.04,6.74,7.75
9,Q10,Acme Corp,Aluminum 6061,Caja,102 × 98 × 47,1,1.26,7.50,140,3 días,110 × 100 × 50,1.49,11.14,16.01


In [12]:
# Modelo realista de costo de mano de obra
# Basado en: tiempo de maquinado estimado × tarifa horaria

# Factores de complejidad por forma (horas base de maquinado)
FACTORES_COMPLEJIDAD = {
    'Caja': 2.5,           # Operaciones simples: 6 caras
    'Cilindro': 1.8,       # Torneado: relativamente simple
    'Cilindro hueco': 3.2  # Torneado + perforación interior: más complejo
}

# Tarifa horaria de maquinado CNC (USD/hora)
TARIFA_HORA = 75  # Tarifa típica para CNC en manufactura

costos_mano_obra = []
precios_totales = []

for idx, row in df_cotizaciones.iterrows():
    forma = row['Forma']
    peso = row['Peso calculado (kg)']
    qty = row['Qty']
    lead_time_dias = int(row['Lead Time'].split()[0])
    costo_material = row['Costo material (USD)']
    precio_IA = row['Precio Total IA']
    
    # Tiempo base según complejidad de la forma
    horas_base = FACTORES_COMPLEJIDAD[forma]
    
    # Ajuste por tamaño/peso (piezas más grandes toman más tiempo)
    # Escala logarítmica para evitar valores extremos
    factor_peso = 1 + (math.log10(max(peso, 0.1)) * 0.3)
    
    # Ajuste por cantidad (economías de escala)
    # Primera pieza toma tiempo completo, las siguientes son más rápidas
    if qty == 1:
        factor_cantidad = 1.0
    else:
        # Primera pieza + resto con 70% del tiempo
        horas_totales = horas_base + (horas_base * 0.7 * (qty - 1))
        factor_cantidad = horas_totales / (horas_base * qty)
    
    # Ajuste por urgencia (rush jobs)
    factor_urgencia = 1.25 if lead_time_dias < 5 else 1.0
    
    # Cálculo final de mano de obra
    horas_efectivas = horas_base * factor_peso * factor_cantidad * factor_urgencia
    costo_mano_obra = horas_efectivas * TARIFA_HORA
    
    costos_mano_obra.append(round(costo_mano_obra, 2))
    
    # Calcular Precio Total con margen 12% superior al de IA
    # Primero calculamos el margen implícito de IA
    costo_total = costo_material + costo_mano_obra
    
    # Margen IA = (Precio IA - Costo Total) / Costo Total
    margen_IA = (precio_IA - costo_total) / costo_total if costo_total > 0 else 0
    
    # Nuestro margen será 12% superior (en puntos porcentuales)
    margen_objetivo = margen_IA + 0.12
    
    # Precio Total = Costo Total * (1 + Margen)
    precio_total = costo_total * (1 + margen_objetivo)
    
    precios_totales.append(round(precio_total, 2))

# Agregar las nuevas columnas
df_cotizaciones['Costo mano de obra (USD)'] = costos_mano_obra
df_cotizaciones['Precio Total (USD)'] = precios_totales

# Calcular y mostrar estadísticas
print("Modelo de costo de mano de obra:")
print(f"- Tarifa horaria: ${TARIFA_HORA}/hora")
print(f"- Horas base por forma: {FACTORES_COMPLEJIDAD}")
print(f"- Ajustes: peso, cantidad (economías de escala), urgencia (+25% si <5 días)")
print(f"\nPrecio Total calculado con margen 12% superior al de IA")

df_cotizaciones

Modelo de costo de mano de obra:
- Tarifa horaria: $75/hora
- Horas base por forma: {'Caja': 2.5, 'Cilindro': 1.8, 'Cilindro hueco': 3.2}
- Ajustes: peso, cantidad (economías de escala), urgencia (+25% si <5 días)

Precio Total calculado con margen 12% superior al de IA


,ID,Cliente,Material,Forma,Dimensiones (mm),Qty,Peso IA (kg),Precio/kg (USD),Precio Total IA,Lead Time,Dimensiones redondeadas,Peso calculado (kg),Costo material (USD),Costo negocio sin mano de obra (USD),Costo mano de obra (USD),Precio Total (USD)
0,Q1,Acme Corp,Aluminum 6061,Caja,102 × 98 × 47,1,1.26,7.50,185,10 días,110 × 100 × 50,1.49,11.14,12.81,197.24,210.01
1,Q2,Acme Corp,Aluminum 6061,Caja,102 × 98 × 47,25,1.26,7.50,185,10 días,110 × 100 × 50,1.49,11.14,10.63,140.44,203.19
2,Q3,BetaTech,Steel 4140,Cilindro,Ø80 × 200,5,2.70,4.75,290,7 días,Ø80 × 200,7.89,37.49,43.11,130.21,310.12
3,Q4,BetaTech,Steel 1018,Cilindro,Ø80 × 200,5,7.90,2.25,285,7 días,Ø80 × 200,7.89,17.76,20.42,130.21,302.76
4,Q5,CanDo Mfg,Titanium,Caja,2 × 2 × 4,1,1.10,62.00,320,14 días,10 × 10 × 10,0.00,0.27,150.32,131.25,335.78
5,Q6,CanDo Mfg,Titanium,Caja,50 × 50 × 100,20,1.10,62.00,145,14 días,50 × 50 × 100,1.11,68.66,73.04,135.89,169.55
6,Q7,DeltaAero,Aluminum 7075,Caja,203 × 152 × 22,50,2.10,15.00,95,3 días,210 × 160 × 30,2.83,42.49,50.69,187.90,122.65
7,Q8,EastMech,Stainless 316,Cilindro hueco,Ø60 (p5) × 150,10,2.00,9.50,175,8 días,Ø60 (p5.0) × 150,1.04,9.85,11.33,176.10,197.31
8,Q9,EastMech,Stainless 304,Cilindro hueco,Ø60 (p5) × 150,10,2.00,6.50,210,8 días,Ø60 (p5.0) × 150,1.04,6.74,7.75,176.10,231.94
9,Q10,Acme Corp,Aluminum 6061,Caja,102 × 98 × 47,1,1.26,7.50,140,3 días,110 × 100 × 50,1.49,11.14,16.01,246.55,170.92


In [13]:
# Análisis de cumplimiento de reglas de negocio con clasificación y impacto económico
# Cada regla incumplida genera una fila separada
import pandas as pd

reglas_incumplidas = []

for idx, row in df_cotizaciones.iterrows():
    id_cotizacion = row['ID']
    peso_ia = row['Peso IA (kg)']
    peso_calculado = row['Peso calculado (kg)']
    precio_ia = row['Precio Total IA']
    costo_material = row['Costo material (USD)']
    costo_mano_obra = row['Costo mano de obra (USD)']
    material = row['Material']
    qty = row['Qty']
    lead_time_dias = int(row['Lead Time'].split()[0])
    
    cotizacion_sin_errores = True
    
    # REGLA 1: Precio debe exceder (costo_material × 1.15) + mano_de_obra
    precio_minimo = (costo_material * 1.15) + costo_mano_obra
    if precio_ia < precio_minimo:
        diferencia = precio_minimo - precio_ia
        impacto = diferencia * qty  # Pérdida potencial por el batch
        reglas_incumplidas.append({
            'ID': id_cotizacion,
            'Cliente': row['Cliente'],
            'Material': material,
            'Qty': qty,
            'Lead Time': row['Lead Time'],
            'Precio IA (USD)': precio_ia,
            'Precio Correcto (USD)': row['Precio Total (USD)'],
            'Regla': 'Regla 1: Precio mínimo',
            'Clasificación': '⚠️ Warning',
            'Descripción': f'Precio insuficiente. Cotizado: ${precio_ia}, Mínimo: ${precio_minimo:.2f} (falta ${diferencia:.2f})',
            'Impacto Económico (USD)': f'${impacto:.2f}',
            'Tipo Impacto': 'Pérdida directa'
        })
        cotizacion_sin_errores = False
    
    # REGLA 2: Peso cotizado debe reflejar dimensiones de STOCK
    tolerancia_peso = 0.011  # Tolerancia de 0.1 kg
    diferencia_peso = abs(peso_ia - peso_calculado)
    if diferencia_peso > tolerancia_peso:
        precio_kg = row['Precio/kg (USD)']
        impacto = diferencia_peso * precio_kg * qty
        clasificacion = '🛑 Crítico' if diferencia_peso > 0.1 else '⚠️ Warning'
        reglas_incumplidas.append({
            'ID': id_cotizacion,
            'Cliente': row['Cliente'],
            'Material': material,
            'Qty': qty,
            'Lead Time': row['Lead Time'],
            'Precio IA (USD)': precio_ia,
            'Precio Correcto (USD)': row['Precio Total (USD)'],
            'Regla': 'Regla 2: Peso correcto',
            'Clasificación': clasificacion,
            'Descripción': f'Peso incorrecto. IA: {peso_ia} kg, Correcto: {peso_calculado} kg (diferencia: {peso_ia - peso_calculado:+.2f} kg)',
            'Impacto Económico (USD)': f'${impacto:.2f}',
            'Tipo Impacto': 'Error en cotización'
        })
        cotizacion_sin_errores = False
    
    # REGLA 3: Titanio - surcharge de $150 USD amortizado
    if 'Titanium' in material:
        surcharge_esperado = 150 / qty
        precio_base_sin_surcharge = (costo_material * 1.15 ) + costo_mano_obra
        if precio_ia < precio_base_sin_surcharge + surcharge_esperado:
            impacto = 150  # Pérdida del surcharge completo
            reglas_incumplidas.append({
                'ID': id_cotizacion,
                'Cliente': row['Cliente'],
                'Material': material,
                'Qty': qty,
                'Lead Time': row['Lead Time'],
                'Precio IA (USD)': precio_ia,
                'Precio Correcto (USD)': row['Precio Total (USD)'],
                'Regla': 'Regla 3: Surcharge Titanio',
                'Clasificación': '🛑 Crítico',
                'Descripción': f'Falta surcharge de Titanio. Esperado: ${surcharge_esperado:.2f}/pc (${150:.2f} total)',
                'Impacto Económico (USD)': f'${impacto:.2f}',
                'Tipo Impacto': 'Costo tooling no recuperado'
            })
            cotizacion_sin_errores = False
    
    # REGLA 4: Descuento por volumen 15-20% en cantidades 10+
    if qty > 10:
        # precio_con_descuento_min = precio_minimo * 0.80
        precio_con_descuento_max = precio_minimo * 0.85  # Descuento mínimo 15%
        
        if precio_ia > precio_con_descuento_max:
            descuento_esperado = precio_minimo * 0.175  # Promedio 17.5%
            impacto = (precio_ia - (precio_minimo - descuento_esperado)) * qty
            reglas_incumplidas.append({
                'ID': id_cotizacion,
                'Cliente': row['Cliente'],
                'Material': material,
                'Qty': qty,
                'Lead Time': row['Lead Time'],
                'Precio IA (USD)': precio_ia,
                'Precio Correcto (USD)': row['Precio Total (USD)'],
                'Regla': 'Regla 4: Descuento volumen',
                'Clasificación': '🛑 Crítico',
                'Descripción': f'No aplicó descuento por volumen. Qty={qty}, esperado 15-20% de descuento',
                'Impacto Económico (USD)': f'${impacto:.2f}',
                'Tipo Impacto': 'Riesgo competitividad'
            })
            cotizacion_sin_errores = False
    
    # REGLA 5: Premium por urgencia +25% si lead time < 5 días
    if lead_time_dias < 5:
        precio_con_premium = precio_minimo * 1.25
        if precio_ia < precio_con_premium * 0.95:  # Tolerancia 5%
            impacto = (precio_con_premium - precio_ia) * qty
            reglas_incumplidas.append({
                'ID': id_cotizacion,
                'Cliente': row['Cliente'],
                'Material': material,
                'Qty': qty,
                'Lead Time': row['Lead Time'],
                'Precio IA (USD)': precio_ia,
                'Precio Correcto (USD)': row['Precio Total (USD)'],
                'Regla': 'Regla 5: Premium urgencia',
                'Clasificación': '🛑 Crítico',
                'Descripción': f'No aplicó premium por urgencia. Lead time: {lead_time_dias} días, esperado +25%',
                'Impacto Económico (USD)': f'${impacto:.2f}',
                'Tipo Impacto': 'Ingreso no capturado'
            })
            cotizacion_sin_errores = False
    
    # Si no hay errores, agregar una fila indicando que está correcta
    if cotizacion_sin_errores:
        reglas_incumplidas.append({
            'ID': id_cotizacion,
            'Cliente': row['Cliente'],
            'Material': material,
            'Qty': qty,
            'Lead Time': row['Lead Time'],
            'Precio IA (USD)': precio_ia,
            'Precio Correcto (USD)': row['Precio Total (USD)'],
            'Regla': 'Sin errores',
            'Clasificación': '✅ Correcto',
            'Descripción': 'Cumple todas las reglas de negocio',
            'Impacto Económico (USD)': '$0.00',
            'Tipo Impacto': 'Sin impacto'
        })

# Crear dataframe de resumen
df_incumplimiento = pd.DataFrame(reglas_incumplidas)

# Estadísticas
total_criticos = len(df_incumplimiento[df_incumplimiento['Clasificación'] == '🛑 Crítico'])
total_warnings = len(df_incumplimiento[df_incumplimiento['Clasificación'] == '⚠️ Warning'])
total_correctos = len(df_incumplimiento[df_incumplimiento['Clasificación'] == '✅ Correcto'])
total_cotizaciones_unicas = df_incumplimiento['ID'].nunique()

# Calcular impacto económico total
df_incumplimiento['Impacto_Num'] = df_incumplimiento['Impacto Económico (USD)'].str.replace('$', '').str.replace(',', '').astype(float)
impacto_economico_total = df_incumplimiento['Impacto_Num'].sum()

# Cotizaciones con al menos un error crítico
cotizaciones_criticas = df_incumplimiento[df_incumplimiento['Clasificación'] == '🛑 Crítico']['ID'].nunique()
cotizaciones_warning = df_incumplimiento[df_incumplimiento['Clasificación'] == '⚠️ Warning']['ID'].nunique()
cotizaciones_correctas = df_incumplimiento[df_incumplimiento['Clasificación'] == '✅ Correcto']['ID'].nunique()

print("="*80)
print("AUDITORÍA DE CUMPLIMIENTO DE REGLAS DE NEGOCIO")
print("Formato: Una fila por cada regla incumplida")
print("="*80)
print(f"\nCotizaciones analizadas: {total_cotizaciones_unicas}")
print(f"  🛑 Con errores críticos: {cotizaciones_criticas} ({cotizaciones_criticas/total_cotizaciones_unicas*100:.1f}%)")
print(f"  ⚠️  Con warnings: {cotizaciones_warning} ({cotizaciones_warning/total_cotizaciones_unicas*100:.1f}%)")
print(f"  ✅ Totalmente correctas: {cotizaciones_correctas} ({cotizaciones_correctas/total_cotizaciones_unicas*100:.1f}%)")
print(f"\nTotal de incumplimientos detectados:")
print(f"  🛑 Críticos: {total_criticos}")
print(f"  ⚠️  Warnings: {total_warnings}")
print(f"  ✅ Correctos: {total_correctos}")
print(f"\n💰 IMPACTO ECONÓMICO TOTAL: ${impacto_economico_total:,.2f} USD")
print("="*80)

# Eliminar columna auxiliar
df_incumplimiento = df_incumplimiento.drop('Impacto_Num', axis=1)

# Configurar pandas para mostrar todo el texto sin truncar
pd.set_option('display.max_colwidth', None)  # Sin límite en el ancho de columnas
pd.set_option('display.max_rows', None)      # Mostrar todas las filas
pd.set_option('display.width', None)         # Sin límite de ancho total
pd.set_option('display.max_columns', None)   # Mostrar todas las columnas

df_incumplimiento

AUDITORÍA DE CUMPLIMIENTO DE REGLAS DE NEGOCIO
Formato: Una fila por cada regla incumplida

Cotizaciones analizadas: 10
  🛑 Con errores críticos: 9 (90.0%)
  ⚠️  Con warnings: 5 (50.0%)
  ✅ Totalmente correctas: 1 (10.0%)

Total de incumplimientos detectados:
  🛑 Críticos: 12
  ⚠️  Warnings: 5
  ✅ Correctos: 1

💰 IMPACTO ECONÓMICO TOTAL: $21,539.08 USD


,ID,Cliente,Material,Qty,Lead Time,Precio IA (USD),Precio Correcto (USD),Regla,Clasificación,Descripción,Impacto Económico (USD),Tipo Impacto
0,Q1,Acme Corp,Aluminum 6061,1,10 días,185,210.01,Regla 1: Precio mínimo,⚠️ Warning,"Precio insuficiente. Cotizado: $185, Mínimo: $210.05 (falta $25.05)",$25.05,Pérdida directa
1,Q1,Acme Corp,Aluminum 6061,1,10 días,185,210.01,Regla 2: Peso correcto,🛑 Crítico,"Peso incorrecto. IA: 1.26 kg, Correcto: 1.49 kg (diferencia: -0.23 kg)",$1.72,Error en cotización
2,Q2,Acme Corp,Aluminum 6061,25,10 días,185,203.19,Regla 2: Peso correcto,🛑 Crítico,"Peso incorrecto. IA: 1.26 kg, Correcto: 1.49 kg (diferencia: -0.23 kg)",$43.12,Error en cotización
3,Q2,Acme Corp,Aluminum 6061,25,10 días,185,203.19,Regla 4: Descuento volumen,🛑 Crítico,"No aplicó descuento por volumen. Qty=25, esperado 15-20% de descuento",$1464.20,Riesgo competitividad
4,Q3,BetaTech,Steel 4140,5,7 días,290,310.12,Regla 2: Peso correcto,🛑 Crítico,"Peso incorrecto. IA: 2.7 kg, Correcto: 7.89 kg (diferencia: -5.19 kg)",$123.26,Error en cotización
5,Q4,BetaTech,Steel 1018,5,7 días,285,302.76,Sin errores,✅ Correcto,Cumple todas las reglas de negocio,$0.00,Sin impacto
6,Q5,CanDo Mfg,Titanium,1,14 días,320,335.78,Regla 2: Peso correcto,🛑 Crítico,"Peso incorrecto. IA: 1.1 kg, Correcto: 0.0 kg (diferencia: +1.10 kg)",$68.20,Error en cotización
7,Q6,CanDo Mfg,Titanium,20,14 días,145,169.55,Regla 1: Precio mínimo,⚠️ Warning,"Precio insuficiente. Cotizado: $145, Mínimo: $214.85 (falta $69.85)",$1396.98,Pérdida directa
8,Q6,CanDo Mfg,Titanium,20,14 días,145,169.55,Regla 3: Surcharge Titanio,🛑 Crítico,Falta surcharge de Titanio. Esperado: $7.50/pc ($150.00 total),$150.00,Costo tooling no recuperado
9,Q7,DeltaAero,Aluminum 7075,50,3 días,95,122.65,Regla 1: Precio mínimo,⚠️ Warning,"Precio insuficiente. Cotizado: $95, Mínimo: $236.76 (falta $141.76)",$7088.18,Pérdida directa


### Parte A — Detección de errores (lo más importante)

Para cada cotización:
- ¿El peso es correcto? Muestra tus cálculos.
    - Los cálculos se encuentran en el archivo Auditoría de cotizaciones.txt. Podemos observar la tabla de arriba para ver si los pesos fueron correctos
- ¿El precio cumple las reglas de negocio?
    - En la tabla de arriba podemos ver en resumen qué cotizaciones incumplieron cuáles reglas de negocio.
- Clasifica cada quote: ✅ Correcto, ⚠️ Warning, 🛑 Crítico
    - En la tabla de arriba se muestra la clasificación de cada cotización
- Estima el impacto en dinero donde sea posible
    - Los impactos estimados se encuentran en la tabla de arriba
    

### Parte B — Análisis de patrones

1. ¿Por qué la tasa de conversión de titanio es solo 8%?
- Mi suposición más fuerte es que es debido al surcharge. Intuyo que los clientes primero querrían ver si la pieza que diseñan encaja bien en la realidad y si la calidad es la que esperan, por lo que primero solo les interesa realizar una pieza. El surcharge aumenta demasiado la manufactura de una sola pieza por lo que es probable que esto los desánime a continuar con el proceso de compra.
2. ¿Por qué los márgenes de la IA son 12% más bajos que los humanos?
- El modelo tiene problemas aplicando todas las reglas de negocio, pareciera que el modelo utilizado es un random forest empleado para valores continuos en vez de clasificación, únicamente pone el resultado más probable seguido de los datos con los que entrenó, no interpola para casos nuevos. El 12% puede ser resultado de un entrenamiento con valores desactualizados, o un batch de cotizaciones que por suerte dió 12%. Para entender mejor este fenómeno necesitamos analizar más profundamente cómo se obtiene este valor. Dado este comportamiento del modelo, es probable que al analizar la dispersión de las predicciones, esta sea sumamente errática y este 12% sea sólo producto de una coincidencia y no de un patrón.
3. ¿Qué pasó en los meses 4-6? ¿Qué le preguntarías al equipo? 
- Es probable que las primeras cotizaciones que realizó el módelo hayan sido para piezas más sencillas. Posiblemente los usuarios primero querían comprobar la fiabilidad de la herramienta con diseños menos críticos y costosos para asegurarse que obtenían sus piezas con la calidad y las especificaciones que indicaron. A medida que la herramienta se fue dando a conocer, llegaron más clientes con diseños más complejos, con diferentes formatos o con errores que el modelo no estaba entrenado para cotizar dando precios más bajos para piezas complejas o muy altos para piesas simlpes que afectaron las tasas de conversión. 

- Al equipo le haría las siguientes preguntas para ver si mi principal suposición es correcta o existen otras razones:
    - ¿Los precios de los materiales y la mano de obra varían dependiendo de la época del año? 
    - ¿Qué tan rápido suben los precios con el tiempo?
    - ¿Con qué frecuencia se reentrena el modelo?
    - ¿De cuándo son los datos con los que se entrenó el módelo?
    - ¿El modelo puede manejar todos los formatos de diseños? ¿Con cuáles falló? ¿Cuáles cotizó aceptablemente?
    - ¿Qué patrones denotan las cotizaciones fallidas y las cotizaciones acertadas?
    - ¿Cuáles son sus suposiciones sobre la causa de esta situación?


### Parte C — Reglas de validación

Escribe **5 reglas** en Python que atrapen estos errores automáticamente.
Para cada regla incluye:
- El código
    - El código de las validaciones se en cuentra en la última celda de código de este notebook.
- Qué errores atrapa
    - Cada validación tiene cubre una regla de negocio especificada en un comentario encima de su código.
- Estimación de tasa de falsos positivos
    - De 
- Propón al menos una fuente de datos externa que mejoraría la validación